In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import time

# Gender selection
gender = input("Select Gender (male/female): ").strip().lower()
if gender not in ['male', 'female']:
    print("Invalid gender selection.")
    exit(1)

# Initialize MediaPipe Pose and Hands
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.78)

# Helper function to overlay PNG images
def overlay_png(image, overlay, position):
    if overlay.shape[2] == 3:  # If the overlay does not have an alpha channel
        overlay = np.dstack((overlay, np.ones((overlay.shape[0], overlay.shape[1]), dtype="uint8") * 255))
    
    y_offset, x_offset = map(int, position)  # Ensure offsets are integers
    y1, y2 = y_offset, y_offset + overlay.shape[0]
    x1, x2 = x_offset, x_offset + overlay.shape[1]

    # Ensure overlay does not exceed image boundaries
    if y1 < 0: y1 = 0
    if y2 > image.shape[0]: y2 = image.shape[0]
    if x1 < 0: x1 = 0
    if x2 > image.shape[1]: x2 = image.shape[1]

    overlay_resized = overlay[0:y2-y1, 0:x2-x1]

    alpha_s = overlay_resized[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    for c in range(0, 3):
        image[y1:y2, x1:x2, c] = (alpha_s * overlay_resized[:, :, c] + alpha_l * image[y1:y2, x1:x2, c])

    return image

# Category paths
categories = ["casual", "formal", "summer", "winter"]
category_buttons = ["casual.png", "formal.png", "summer.png", "winter.png"]
category_folder_path = f"{gender}/"

# Initialize video capture from camera
cap = cv2.VideoCapture(0)

# Set the window size
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)

# Load category buttons
category_button_images = []
for button in category_buttons:
    button_path = f"Category_buttons/{button}"
    button_img = cv2.imread(button_path, cv2.IMREAD_UNCHANGED)
    if button_img is not None:
        button_img = cv2.resize(button_img, (180, 100))
        category_button_images.append(button_img)
    else:
        print(f"Failed to load button image: {button_path}")

category_selected = False

while True:
    success, img = cap.read()
    if not success:
        print("Failed to read from camera.")
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)

    if not category_selected:
        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                hand_landmarks_x = [lm.x * img.shape[1] for lm in hand_landmarks.landmark]
                hand_landmarks_y = [lm.y * img.shape[0] for lm in hand_landmarks.landmark]

                # Check for category selection
                for i, button_img in enumerate(category_button_images):
                    if i < 2:
                        # Left side buttons (casual, formal)
                        x_pos = 50
                        y_pos = 100 + i * 150
                    else:
                        # Right side buttons (summer, winter)
                        x_pos = img.shape[1] - 150
                        y_pos = 100 + (i - 2) * 150

                    if (x_pos < hand_landmarks_x[8] < x_pos + 100 and
                            y_pos < hand_landmarks_y[8] < y_pos + 100):
                        selected_category = categories[i]
                        category_folder_path = f"{gender}/{selected_category}/"
                        category_selected = True
                        print(f"Selected category: {selected_category}")
                        break

                if category_selected:
                    break

        # Display category buttons
        for i, button_img in enumerate(category_button_images):
            if i < 2:
                x_pos = 50
                y_pos = 100 + i * 150
            else:
                x_pos = img.shape[1] - 180
                y_pos = 100 + (i - 2) * 150
            img = overlay_png(img, button_img, (y_pos, x_pos))

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if not category_selected:
            continue

    shirtFolderPath = category_folder_path
    if not os.path.exists(shirtFolderPath):
        print(f"Shirt folder path '{shirtFolderPath}' does not exist.")
        category_selected = False
        continue

    # Filter to get only valid image files
    valid_image_extensions = [".png", ".jpg", ".jpeg"]
    listShirts = [f for f in os.listdir(shirtFolderPath) if os.path.splitext(f)[1].lower() in valid_image_extensions]

    if len(listShirts) == 0:
        print(f"No valid shirt images found in the directory '{shirtFolderPath}'.")
        category_selected = False
        continue

    print(f"Found {len(listShirts)} valid shirts in the directory '{shirtFolderPath}'.")

    fixedRatio = 370 / 200  # widthOfShirt/widthOfPoint11to12
    shirtRatioHeightWidth = 581 / 440
    imageNumber = 0

    # Thumbnail settings
    thumbnail_width = 100
    thumbnail_height = int(thumbnail_width * shirtRatioHeightWidth)
    thumbnail_gap = 20
    thumbnails = []

    # Load thumbnails
    for shirt in listShirts:
        imgShirtPath = os.path.join(shirtFolderPath, shirt)
        imgShirt = cv2.imread(imgShirtPath, cv2.IMREAD_UNCHANGED)
        if imgShirt is not None:
            imgShirtThumbnail = cv2.resize(imgShirt, (thumbnail_width, thumbnail_height))
            thumbnails.append(imgShirtThumbnail)

    # Load buttons
    exit_button = cv2.imread("button/exit-button-icon-18.png", cv2.IMREAD_UNCHANGED)
    picture_click_button = cv2.imread("button/pngtree-blue-camera-button-cirlce-png-image_4255760-removebg-preview.png", cv2.IMREAD_UNCHANGED)
    back_button = cv2.imread("button/button.png", cv2.IMREAD_UNCHANGED)  # Assume you have a back button image
    button_size = 100

    # Resize buttons
    exit_button = cv2.resize(exit_button, (button_size, button_size))
    picture_click_button = cv2.resize(picture_click_button, (button_size, button_size))
    back_button = cv2.resize(back_button, (button_size, button_size))

    selected_shirt_index = 0
    scroll_offset = 0
    scroll_velocity = 0
    scroll_friction = 0.9
    select_timer = 0
    selection_delay = 2  # Seconds required to select a shirt
    last_picture_time = 0

    # Path for saving pictures
    picture_save_path = "shirts/pictures"
    os.makedirs(picture_save_path, exist_ok=True)

    while True:
        success, img = cap.read()
        if not success:
            print("Failed to read from camera.")
            break

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results_pose = pose.process(img_rgb)
        results_hands = hands.process(img_rgb)

        current_time = time.time()

        if results_pose.pose_landmarks:
            lmList = results_pose.pose_landmarks.landmark
            lm11 = [int(lmList[11].x * img.shape[1]), int(lmList[11].y * img.shape[0])]
            lm12 = [int(lmList[12].x * img.shape[1]), int(lmList[12].y * img.shape[0])]
            lm23 = [int(lmList[23].x * img.shape[1]), int(lmList[23].y * img.shape[0])]
            # Debugging pose landmarks
            print(f"lm23: {lm23}, lm11: {lm11}, lm12: {lm12}")

            widthOfShirt = int((lm11[0] - lm12[0]) * fixedRatio)
            
            if widthOfShirt > 0:  # Ensure the width is positive
                try:
                    imgShirtPath = os.path.join(shirtFolderPath, listShirts[selected_shirt_index])
                    print(f"Loading shirt image from path: {imgShirtPath}")
                    imgShirt = cv2.imread(imgShirtPath, cv2.IMREAD_UNCHANGED)
                    if imgShirt is not None:
                        imgShirt = cv2.resize(imgShirt, (widthOfShirt, int(widthOfShirt * shirtRatioHeightWidth)))
                        currentScale = widthOfShirt / imgShirt.shape[1]
                        # Calculate the midpoint between lm11 and lm12
                        midpoint_x = (lm11[0] + lm12[0]) // 2
                        midpoint_y = (lm11[1] + lm12[1]) // 2
                        
                        # Calculate the position to place the shirt overlay
                        x_offset = midpoint_x - (imgShirt.shape[1] // 2)
                        y_offset = lm11[1] - 110 # Adjust as needed to position the shirt correctly vertically
                        
                        # Debugging shirt overlay position
                        print(f"Shirt Position: (x: {x_offset}, y: {y_offset}), Size: {imgShirt.shape}")
                        
                        # Overlay the shirt on the image
                        img = overlay_png(img, imgShirt, (y_offset, x_offset))
                    else:
                        print(f"Failed to load shirt image: {imgShirtPath}")
                except Exception as e:
                    print(f"Error loading or resizing shirt image: {e}")
                    pass

        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                hand_landmarks_x = [lm.x * img.shape[1] for lm in hand_landmarks.landmark]
                hand_landmarks_y = [lm.y * img.shape[0] for lm in hand_landmarks.landmark]

                # Debugging hand landmark coordinates
                print(f"Hand Landmark X: {hand_landmarks_x}")
                print(f"Hand Landmark Y: {hand_landmarks_y}")

                # Check if hand is selecting a thumbnail
                selecting = False
                for i, thumbnail in enumerate(thumbnails):
                    y_position = thumbnail_gap + i * (thumbnail_height + thumbnail_gap) - scroll_offset
                    if (thumbnail_gap < hand_landmarks_x[8] < thumbnail_gap + thumbnail_width and
                            y_position < hand_landmarks_y[8] < y_position + thumbnail_height):
                        selecting = True
                        if select_timer == 0:
                            select_timer = current_time
                        elif current_time - select_timer >= selection_delay:
                            selected_shirt_index = i
                            print(f"Selected shirt index: {selected_shirt_index}")
                            select_timer = 0  # Reset the timer after selection
                        break

                if not selecting:
                    select_timer = 0  # Reset timer if not selecting

                # Draw selection progress bar
                if selecting:
                    progress_length = int((current_time - select_timer) / selection_delay * thumbnail_width)
                    progress_length = max(0, min(progress_length, thumbnail_width))  # Ensure progress_length is within valid bounds
                    cv2.rectangle(img, (thumbnail_gap, img.shape[0] - 30), 
                                  (thumbnail_gap + progress_length, img.shape[0] - 10), 
                                  (0, 255, 0), -1)
                # Check for scrolling gesture
                if hand_landmarks_x[8] < thumbnail_gap:
                    # Calculate the desired scroll offset based on hand position
                    desired_scroll = (len(thumbnails) * (thumbnail_height + thumbnail_gap) - img.shape[0]) * (hand_landmarks_y[8] / img.shape[0])
                    scroll_velocity += (desired_scroll - scroll_offset) * 0.05  # Update scroll velocity based on hand position

                # Check for exit gesture
                if (img.shape[1] - button_size < hand_landmarks_x[8] < img.shape[1] and
                        0 < hand_landmarks_y[8] < button_size):
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()
                    
                # Check for back gesture
                if (img.shape[0] - button_size < hand_landmarks_y[8] < img.shape[0] and
                        img.shape[1] - button_size < hand_landmarks_x[8] < img.shape[1]):
                    print("Back button clicked.")
                    category_selected = False
                    break

                # Check for picture click gesture
                if (img.shape[1] - button_size < hand_landmarks_x[8] < img.shape[1] and
                        img.shape[0] // 2 - button_size // 2 < hand_landmarks_y[8] < img.shape[0] // 2 + button_size // 2):
                    if current_time - last_picture_time > 1:  # Avoid multiple pictures in quick succession
                        picture_name = f"shirt_{int(current_time)}.png"
                        picture_path = os.path.join(picture_save_path, picture_name)
                        cv2.imwrite(picture_path, img)
                        print(f"Picture saved: {picture_path}")
                        last_picture_time = current_time
                        # Blink effect
                        img = np.zeros_like(img)
                        cv2.imshow("Image", img)
                        cv2.waitKey(100)

        if not category_selected:
            # Break the loop to go back to category selection screen
            break

        # Apply scroll velocity and friction
        scroll_offset += scroll_velocity
        scroll_velocity *= scroll_friction

        # Ensure scroll_offset does not exceed the boundaries
        max_scroll_offset = max(0, len(thumbnails) * (thumbnail_height + thumbnail_gap) - img.shape[0])
        scroll_offset = min(max(scroll_offset, 0), max_scroll_offset)

        # Display thumbnails on the left side of the camera feed
        for i, thumbnail in enumerate(thumbnails):
            y_position = int(thumbnail_gap + i * (thumbnail_height + thumbnail_gap) - scroll_offset)
            if 0 <= y_position + thumbnail_height < img.shape[0]:  # Ensure thumbnail fits within image bounds
                img = overlay_png(img, thumbnail, (y_position, thumbnail_gap))

        # Overlay buttons
        img = overlay_png(img, exit_button, (0, img.shape[1] - button_size))
        img = overlay_png(img, picture_click_button, (img.shape[0] // 2 - button_size // 2, img.shape[1] - button_size))
        img = overlay_png(img, back_button, (img.shape[0] - button_size, img.shape[1] - button_size))

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Select Gender (male/female):  male


C:\Users\AJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Selected category: formal
Found 4 valid shirts in the directory 'male/formal/'.
lm23: [795, 1386], lm11: [851, 715], lm12: [363, 706]
Loading shirt image from path: male/formal/10-removebg-preview.png
Shirt Position: (x: 156, y: 605), Size: (1191, 902, 4)
Hand Landmark X: [110.83662986755371, 186.0453987121582, 232.3581314086914, 256.71260833740234, 259.3776321411133, 156.4858913421631, 155.7573127746582, 151.25537872314453, 144.45284843444824, 122.42669105529785, 124.5429801940918, 123.41094970703125, 120.64615249633789, 86.60076141357422, 92.56912231445312, 98.19202423095703, 101.58761024475098, 48.030290603637695, 55.64692974090576, 64.2253589630127, 74.31084632873535]
Hand Landmark Y: [658.551836013794, 608.2395172119141, 527.7015781402588, 464.66726303100586, 414.2470979690552, 445.1424551010132, 366.49184703826904, 316.9021797180176, 275.0900602340698, 442.09529399871826, 354.169864654541, 296.42396450042725, 248.59957695007324, 447.16694355010986, 362.7867078781128, 310.62945842